# Patient Cohort Building with Unstructured Data: Entity Extraction

In this notebook, we extract clinical and medical entities from texts to create a Knowledge Graph (KG) using pre-trained models in [Spark NLP relation extraction models for healthcare](https://www.johnsnowlabs.com/databricks/?utm_term=sparknlp&utm_campaign=Search+%7C+Spark+NLP&utm_source=adwords&utm_medium=ppc&hsa_acc=7272492311&hsa_cam=12543136013&hsa_grp=121056973604&hsa_ad=605485254464&hsa_src=g&hsa_tgt=kwd-1243265465686&hsa_kw=sparknlp&hsa_mt=p&hsa_net=adwords&hsa_ver=3&gclid=Cj0KCQiAmaibBhCAARIsAKUlaKRjPen9d1iGLcnRo3Ep10euMmW8dd5HuwERjTbbgyaOcNYrwaAeu8caAvmmEALw_wcB).
In the first step of this workflow, we use pre-trained models to extract the entities and their relationships and in the next step we create a KG.

In [0]:
import json
import os

from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel,Pipeline
from pyspark.sql import functions as F
from pyspark.sql.types import *

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_colwidth",100)

print('sparknlp.version : ',sparknlp.version())
print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
<command-713863765819899> in <cell line: 9>()
      7 from pyspark.sql.types import *
      8 
----> 9 from sparknlp.annotator import *
     10 from sparknlp_jsl.annotator import *
     11 from sparknlp.base import *

/databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py in import_patch(name, globals, locals, fromlist, level)
    169             # Import the desired module. If you’re seeing this while debugging a failed import,
    170             # look at preceding stack frames for relevant error information.
--> 171             original_result = python_builtin_import(name, globals, locals, fromlist, level)
    172 
    173             is_root_import = thread_local._nest_level == 1

ModuleNotFoundError: No module named 'sparknlp'

In [0]:
spark._jvm.com.johnsnowlabs.util.start.registerListenerAndStartRefresh()

## Download Medical Dataset

In this notebook, we will use synthetic medical records in csv format.

In [0]:
notes_path='/FileStore/HLS/jsl_kg/data/'
delta_path='/FileStore/HLS/jsl_kg/delta/jsl/'

dbutils.fs.mkdirs(notes_path)
os.environ['notes_path']=f'/dbfs{notes_path}'

In [0]:
%sh
cd $notes_path
wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/databricks/python/healthcare_case_studies/data/data.csv

In [0]:
display(dbutils.fs.ls(f'{notes_path}/'))

path,name,size
dbfs:/FileStore/HLS/jsl_kg/data/data.csv,data.csv,1454309
dbfs:/FileStore/HLS/jsl_kg/data/data.csv.1,data.csv.1,1454309
dbfs:/FileStore/HLS/jsl_kg/data/data.csv.10,data.csv.10,1454309
dbfs:/FileStore/HLS/jsl_kg/data/data.csv.11,data.csv.11,1454309
dbfs:/FileStore/HLS/jsl_kg/data/data.csv.12,data.csv.12,1454309
dbfs:/FileStore/HLS/jsl_kg/data/data.csv.13,data.csv.13,1454309
dbfs:/FileStore/HLS/jsl_kg/data/data.csv.2,data.csv.2,1454309
dbfs:/FileStore/HLS/jsl_kg/data/data.csv.3,data.csv.3,1454309
dbfs:/FileStore/HLS/jsl_kg/data/data.csv.4,data.csv.4,1454309
dbfs:/FileStore/HLS/jsl_kg/data/data.csv.5,data.csv.5,1454309


## Read Data and Write to Bronze Delta Layer

There are 965 clinical records stored in delta table. We read the data and write the records into bronze delta tables.

In [0]:
df = spark.createDataFrame(pd.read_csv(f'/dbfs{notes_path}/data.csv', sep=';'))
df.limit(20).display()

subject_id date text gender dateOfBirth 19823 2167-02-25 Admission Date: [**2167-2-16**] Discharge Date: [**2167-2-24**]

Date of Birth: [**2099-5-5**] Sex: F

Service: [**Hospital Unit Name 196**]

CHIEF COMPLAINT: Shortness of breath, cough, and fever.

HISTORY OF PRESENT ILLNESS: The patient is a 67 -year-old
Russian speaking female with a past medical history
significant for diabetes type II, congestive heart failure of
unknown etiology, and hypertension. The patient presents
with a three day history of progressively worsening shortness
of breath and dyspnea on exertion, wheezing, nonproductive
cough, and fever to 102 F on the day prior to admission. Per
patient's husband, she denies any nausea or vomiting, chills
at night, night sweats, or chest pain. She denies diarrhea.
She has been constipated. The patient does have paroxysmal
nocturnal dyspnea and two pillow orthopnea. The patient
denies dysuria. Fingersticks at home have been running
approximately 200 to 270's. The patient denies any sick
contacts.

On the morning of presentation, the patient was noted to be
more lethargic by her husband. [**Name (NI) **] report, her oxygen
saturation upon arrival of the EMS, was in the 80's. The
patient was placed on 100% nonrebreather and arrived at [**Hospital3 **] - [**Hospital **] [**First Name (Titles) **] [**Last Name (Titles) **] where she was noted to be
wheezing on examination. She was given Albuterol and
Atrovent nebulizers with improvement of her oxygen saturation
from 90% to 94%, also on 100% nonrebreather. She was also
administered 40 mg of IV Lasix times two with diuresis of
approximately one liter. The patient denied any chest pain
throughout her entire presentation.

PAST MEDICAL HISTORY:
1. Type II diabetes mellitus.
2. Morbid obesity.
3. Hypertension.
4. Congestive heart failure of unclear etiology with normal
[**Name (NI) 20679**] systolic function. Question of left ventricular
hypertrophy on prior echocardiogram.
5. Stasis dermatitis in bilateral lower extremities.
6. No history of coronary artery disease.
7. Restrictive lung disease, believed to be secondary to
morbid obesity. The patient does have a home O2 requirement
of approximately 2.0 to 2.5 liters during the day time.
8. Presumptive obstructive sleep apnea.

ALLERGIES: The patient has no known drug allergies.

ADMITTING MEDICATIONS: 1) Amaryl 2.0 mg po bid, 2)
Glucophage 1,000 mg po bid, 3) Singulair 10 mg po q day, 4)
Zocor 10 mg po q day, 5) Hyzaar 50/12.5 q day, 6) Atenolol 25
mg po q day, 7) Avandia 8.0 mg po bid.

SOCIAL HISTORY: The patient denies any tobacco use. She
lives with her husband, no alcohol use. She is gravida II,
para II.

FAMILY HISTORY: Negative for cancers. Paternal grandmother
with diabetes mellitus and maternal aunt with coronary artery
disease.

PRIOR STUDIES: Echocardiogram of 12/99 revealed mild left
axis deviation, mild mitral regurgitation, and mild pulmonary
artery systolic hypertension. Exercise tolerance test MIBI
was without angina, no ischemic changes, left ventricular
ejection fraction was estimated at 64% with normal wall
motion.

PHYSICAL EXAMINATION: Temperature on presentation was 99.1
F, pulse is 84, blood pressure was 116/50, breathing at a
rate of 19, 96% on 100% nonrebreather. Weight was
approximately 286 pounds. In general, the patient was alert
and oriented times three, on 100% nonrebreather, in moderate
respiratory distress. Head, eyes, ears, nose, and throat:
pupils are equal, round, and reactive to light and
accommodation bilaterally, oropharynx is clear. Neck is
without lymphadenopathy, there is not any assessable jugular
venous pulse. Chest examination reveals diffuse inspiratory
and expiratory wheezes and crackles with a prolonged
expiratory phase. Cardiovascular examination is of regular
rate and rhythm without evidence of murmurs, rubs, or
gallops.

Abdomen is obese without tenderness, guarding, or distention,
there are normoactive bowel sounds. There is no suprapubic
tenderness, no costovertebral angle

In [0]:
df.write.format('delta').mode('overwrite').save(f'{delta_path}/bronze/dataset')
display(dbutils.fs.ls(f'{delta_path}/bronze/dataset'))

path,name,size
dbfs:/FileStore/HLS/jsl_kg/delta/jsl/bronze/dataset/_delta_log/,_delta_log/,0
dbfs:/FileStore/HLS/jsl_kg/delta/jsl/bronze/dataset/part-00000-00239749-0900-4b3e-b923-8994b2c8d947-c000.snappy.parquet,part-00000-00239749-0900-4b3e-b923-8994b2c8d947-c000.snappy.parquet,28005
dbfs:/FileStore/HLS/jsl_kg/delta/jsl/bronze/dataset/part-00000-166696aa-db83-4dba-9108-71e9a2854429-c000.snappy.parquet,part-00000-166696aa-db83-4dba-9108-71e9a2854429-c000.snappy.parquet,28005
dbfs:/FileStore/HLS/jsl_kg/delta/jsl/bronze/dataset/part-00000-18110c06-c341-4a39-b349-3214d8616db7-c000.snappy.parquet,part-00000-18110c06-c341-4a39-b349-3214d8616db7-c000.snappy.parquet,28005
dbfs:/FileStore/HLS/jsl_kg/delta/jsl/bronze/dataset/part-00000-31ac54a2-4565-472e-87db-d09d5225d6d9-c000.snappy.parquet,part-00000-31ac54a2-4565-472e-87db-d09d5225d6d9-c000.snappy.parquet,28005
dbfs:/FileStore/HLS/jsl_kg/delta/jsl/bronze/dataset/part-00000-37f7f523-7de7-4959-b052-0628124c2d29-c000.snappy.parquet,part-00000-37f7f523-7de7-4959-b052-0628124c2d29-c000.snappy.parquet,28005
dbfs:/FileStore/HLS/jsl_kg/delta/jsl/bronze/dataset/part-00000-58d36867-3aab-47f6-a494-b6d3395e3609-c000.snappy.parquet,part-00000-58d36867-3aab-47f6-a494-b6d3395e3609-c000.snappy.parquet,28005
dbfs:/FileStore/HLS/jsl_kg/delta/jsl/bronze/dataset/part-00000-63dcf0f8-f1f7-4701-82f6-25fd8c2cec40-c000.snappy.parquet,part-00000-63dcf0f8-f1f7-4701-82f6-25fd8c2cec40-c000.snappy.parquet,28005
dbfs:/FileStore/HLS/jsl_kg/delta/jsl/bronze/dataset/part-00000-a771aedc-137d-4f48-9617-7ace1ab38906-c000.snappy.parquet,part-00000-a771aedc-137d-4f48-9617-7ace1ab38906-c000.snappy.parquet,28005
dbfs:/FileStore/HLS/jsl_kg/delta/jsl/bronze/dataset/part-00000-c6d0dcb4-2abe-46dd-a39f-4e4f9ca01fa9-c000.snappy.parquet,part-00000-c6d0dcb4-2abe-46dd-a39f-4e4f9ca01fa9-c000.snappy.parquet,28005


## Posology RE Pipeline

### Posology Relation Extraction

Posology relation extraction pretrained model supports the following relatios:

DRUG-DOSAGE  
DRUG-FREQUENCY  
DRUG-ADE (Adversed Drug Events)  
DRUG-FORM  
DRUG-ROUTE  
DRUG-DURATION  
DRUG-REASON  
DRUG=STRENGTH  

The model has been validated against the posology dataset described in (Magge, Scotch, & Gonzalez-Hernandez, 2018).

| Relation | Recall | Precision | F1 | F1 (Magge, Scotch, & Gonzalez-Hernandez, 2018) |
| --- | --- | --- | --- | --- |
| DRUG-ADE | 0.66 | 1.00 | **0.80** | 0.76 |
| DRUG-DOSAGE | 0.89 | 1.00 | **0.94** | 0.91 |
| DRUG-DURATION | 0.75 | 1.00 | **0.85** | 0.92 |
| DRUG-FORM | 0.88 | 1.00 | **0.94** | 0.95* |
| DRUG-FREQUENCY | 0.79 | 1.00 | **0.88** | 0.90 |
| DRUG-REASON | 0.60 | 1.00 | **0.75** | 0.70 |
| DRUG-ROUTE | 0.79 | 1.00 | **0.88** | 0.95* |
| DRUG-STRENGTH | 0.95 | 1.00 | **0.98** | 0.97 |


*Magge, Scotch, Gonzalez-Hernandez (2018) collapsed DRUG-FORM and DRUG-ROUTE into a single relation.

In [0]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("documents")

sentencer = SentenceDetector()\
    .setInputCols(["documents"])\
    .setOutputCol("sentences")

tokenizer = sparknlp.annotators.Tokenizer()\
    .setInputCols(["sentences"])\
    .setOutputCol("tokens")

words_embedder = WordEmbeddingsModel()\
    .pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

pos_tagger = PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("pos_tags")

posology_ner = MedicalNerModel()\
    .pretrained("ner_posology", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ners")   

posology_ner_converter = NerConverterInternal() \
    .setInputCols(["sentences", "tokens", "ners"]) \
    .setOutputCol("ner_chunks")

dependency_parser = DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentences", "pos_tags", "tokens"])\
    .setOutputCol("dependencies")

reModel = RelationExtractionModel()\
    .pretrained("posology_re")\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("posology_relations")\
    .setMaxSyntacticDistance(4)

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    posology_ner,
    posology_ner_converter,
    dependency_parser,
    reModel
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]
ner_posology download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
results = model.transform(df)
results.printSchema()

root
-- subject_id: long (nullable = true)
-- date: string (nullable = true)
-- text: string (nullable = true)
-- gender: string (nullable = true)
-- dateOfBirth: string (nullable = true)
-- documents: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- sentences: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- tokens: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- embeddings: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- pos_tags: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- ners: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- ner_chunks: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- dependencies: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- posology_relations: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: 

In [0]:
result_df = results.select('subject_id','date',F.explode(F.arrays_zip(results.posology_relations.result, results.posology_relations.metadata)).alias("cols")) \
                   .select('subject_id','date',F.expr("cols['0']").alias("relation"),
                                               F.expr("cols['1']['entity1']").alias("entity1"),
                                               F.expr("cols['1']['entity1_begin']").alias("entity1_begin"),
                                               F.expr("cols['1']['entity1_end']").alias("entity1_end"),
                                               F.expr("cols['1']['chunk1']").alias("chunk1"),
                                               F.expr("cols['1']['entity2']").alias("entity2"),
                                               F.expr("cols['1']['entity2_begin']").alias("entity2_begin"),
                                               F.expr("cols['1']['entity2_end']").alias("entity2_end"),
                                               F.expr("cols['1']['chunk2']").alias("chunk2"),
                                               F.expr("cols['1']['confidence']").alias("confidence"))
result_df.limit(20).display()

subject_id,date,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
19823,2167-02-25,DRUG-FORM,DRUG,1391,1399,Albuterol,FORM,1414,1423,nebulizers,1.0
19823,2167-02-25,DRUG-FORM,DRUG,1405,1412,Atrovent,FORM,1414,1423,nebulizers,1.0
19823,2167-02-25,STRENGTH-DRUG,STRENGTH,1539,1543,40 mg,DRUG,1551,1555,Lasix,1.0
19823,2167-02-25,ROUTE-DRUG,ROUTE,1548,1549,IV,DRUG,1551,1555,Lasix,1.0
19823,2167-02-25,DRUG-STRENGTH,DRUG,2336,2341,Amaryl,STRENGTH,2343,2348,2.0 mg,1.0
19823,2167-02-25,DRUG-ROUTE,DRUG,2336,2341,Amaryl,ROUTE,2350,2351,po,1.0
19823,2167-02-25,DRUG-FREQUENCY,DRUG,2336,2341,Amaryl,FREQUENCY,2353,2355,bid,1.0
19823,2167-02-25,DRUG-STRENGTH,DRUG,2336,2341,Amaryl,STRENGTH,2372,2379,"1,000 mg",1.0
19823,2167-02-25,DRUG-FREQUENCY,DRUG,2336,2341,Amaryl,FREQUENCY,2384,2386,bid,1.0
19823,2167-02-25,STRENGTH-DRUG,STRENGTH,2343,2348,2.0 mg,DRUG,2361,2370,Glucophage,1.0


## RxNorm Code Extraction From Re_Results

In [0]:
# drug + strength or form
from pyspark.sql.functions import when, col

result_df = (
  result_df.withColumn('rx_text', when((F.col('entity1')=='DRUG') & ((F.col('entity2')=='FORM') | (F.col('entity2')=='STRENGTH') | (F.col('entity2')=='DOSAGE') ), F.concat(F.col('chunk1'),F.lit(' '), F.col('chunk2')))
 .when( ((F.col('entity1')=='FORM') | (F.col('entity1')=='STRENGTH') | (F.col('entity1')=='DOSAGE') ) & (F.col('entity2')=='DRUG'), F.concat(F.col('chunk2'),F.lit(' '), F.col('chunk1')))
 .when( (F.col('entity1')=='DRUG') & ((F.col('entity2')!='FORM') & (F.col('entity2')!='STRENGTH') & (F.col('entity2')!='DOSAGE') ), F.col('chunk1'))
 .when( (F.col('entity2')=='DRUG') & ((F.col('entity1')!='FORM') & (F.col('entity1')!='STRENGTH') & (F.col('entity1')!='DOSAGE') ), F.col('chunk2'))
                   .otherwise(F.lit(' '))
                   )
)

result_df.display(20,70)

subject_id,date,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence,rx_text
19823,2167-02-25,DRUG-FORM,DRUG,1391,1399,Albuterol,FORM,1414,1423,nebulizers,1.0,Albuterol nebulizers
19823,2167-02-25,DRUG-FORM,DRUG,1405,1412,Atrovent,FORM,1414,1423,nebulizers,1.0,Atrovent nebulizers
19823,2167-02-25,STRENGTH-DRUG,STRENGTH,1539,1543,40 mg,DRUG,1551,1555,Lasix,1.0,Lasix 40 mg
19823,2167-02-25,ROUTE-DRUG,ROUTE,1548,1549,IV,DRUG,1551,1555,Lasix,1.0,Lasix
19823,2167-02-25,DRUG-STRENGTH,DRUG,2336,2341,Amaryl,STRENGTH,2343,2348,2.0 mg,1.0,Amaryl 2.0 mg
19823,2167-02-25,DRUG-ROUTE,DRUG,2336,2341,Amaryl,ROUTE,2350,2351,po,1.0,Amaryl
19823,2167-02-25,DRUG-FREQUENCY,DRUG,2336,2341,Amaryl,FREQUENCY,2353,2355,bid,1.0,Amaryl
19823,2167-02-25,DRUG-STRENGTH,DRUG,2336,2341,Amaryl,STRENGTH,2372,2379,"1,000 mg",1.0,"Amaryl 1,000 mg"
19823,2167-02-25,DRUG-FREQUENCY,DRUG,2336,2341,Amaryl,FREQUENCY,2384,2386,bid,1.0,Amaryl
19823,2167-02-25,STRENGTH-DRUG,STRENGTH,2343,2348,2.0 mg,DRUG,2361,2370,Glucophage,1.0,Glucophage 2.0 mg


In [0]:
documentAssembler = DocumentAssembler()\
      .setInputCol("rx_text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings.pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sentence_embeddings")
    
rxnorm_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm_augmented","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sentence_embeddings"]) \
      .setOutputCol("rxnorm_code")\
      .setDistanceFunction("EUCLIDEAN")

rxnorm_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        rxnorm_resolver])

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]
sbiobertresolve_rxnorm_augmented download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

In [0]:
rxnorm_results = rxnorm_pipelineModel.transform(result_df)
rxnorm_result = rxnorm_results.select('subject_id','date', 'relation', 'entity1', 'entity1_begin','entity1_end',  'chunk1', 'entity2', 'entity2_begin', 'entity2_end', 
                                         'chunk2', 'confidence', 'rx_text', 
                                         F.explode(F.arrays_zip(rxnorm_results.ner_chunk.result, 
                                                                rxnorm_results.ner_chunk.metadata, 
                                                                rxnorm_results.rxnorm_code.result, 
                                                                rxnorm_results.rxnorm_code.metadata)).alias("cols")) \
                                     .select('subject_id','date', 'relation', 'entity1', 'entity1_begin','entity1_end',  'chunk1', 'entity2', 'entity2_begin', 'entity2_end',
                                             'chunk2', 'confidence', 'rx_text',
                                             F.expr("cols['1']['sentence']").alias("sent_id"),
                                             F.expr("cols['0']").alias("ner_chunk"),
                                             F.expr("cols['1']['entity']").alias("entity"), 
                                             F.expr("cols['2']").alias('rxnorm_code'),
                                             F.expr("cols['3']['all_k_results']").alias("all_codes"),
                                             F.expr("cols['3']['all_k_resolutions']").alias("resolutions"))
rxnorm_result.limit(20).display()

subject_id,date,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence,rx_text,sent_id,ner_chunk,entity,rxnorm_code,all_codes,resolutions
19823,2167-02-25,DRUG-FORM,DRUG,1391,1399,Albuterol,FORM,1414,1423,nebulizers,1.0,Albuterol nebulizers,0,Albuterol nebulizers,null,2108226,2108226:::1154602:::370790:::1154603:::2108233:::2108255:::2108276:::745678:::1163444:::2108246:::2108507:::1154987:::2108228:::437534:::1008406:::2108280:::2108463:::1154598:::2108451:::2108291:::2108552:::377207:::1154604:::1156070,albuterol Inhalation Solution:::albuterol Inhalant Product:::albuterol Injectable Solution:::albuterol Injectable Product:::albuterol Inhalation Powder:::metaproterenol Inhalation Solution [Alupent]:::arformoterol Inhalation Solution:::albuterol Metered Dose Inhaler:::levalbuterol Inhalant Product:::albuterol Inhalation Solution [Airet]:::sodium chloride Inhalation Solution [Nebusal]:::atropine Inhalant Product:::albuterol Inhalation Solution [Accuneb]:::albuterol / ambroxol Oral Solution:::Albuterol / Ambroxol:::atropine Inhalation Solution:::isoflurane Inhalation Solution [Attane]:::albuterol / ipratropium Inhalant Product:::ipratropium Inhalation Solution [Atrovent]:::bitolterol Inhalation Solution:::tobramycin Inhalation Solution [Tobi]:::acebutolol Injectable Solution:::albuterol Oral Liquid Product:::budesonide Inhalant Product
19823,2167-02-25,DRUG-FORM,DRUG,1405,1412,Atrovent,FORM,1414,1423,nebulizers,1.0,Atrovent nebulizers,0,Atrovent nebulizers,null,2108451,2108451:::1173573:::379767:::1173576:::2463732:::1945043:::1172634:::1171309:::363357:::1184866:::1170108:::1593875:::1941535:::1547449:::1547653:::2108255:::1367507:::2056699:::1795589:::2108507:::1170006:::1170113:::1176096:::2108246,ipratropium Inhalation Solution [Atrovent]:::Atrovent Inhalant Product:::Atrovent Autohaler:::Atrovent Nasal Product:::epoetin alfa Injectable Solution [Retacrit]:::Trelegy Inhalant Product:::Fortical Inhalant Product:::Advair Inhalant Product:::alefacept Injectable Solution [Amevive]:::Suprane Inhalant Product:::Arduan Injectable Product:::Altamist Inhalant Product:::Armonair Inhalant Product:::Nutrilyte Injectable Product:::Arnuity Inhalant Product:::metaproterenol Inhalation Solution [Alupent]:::Adasuve Inhalant Product:::Ajovy Injectable Product:::Nasoflow Inhalant Product:::sodium chloride Inhalation Solution [Nebusal]:::Alupent Inhalant Product:::Arfonad Injectable Product:::Haldol Injectable Product:::albuterol Inhalation Solution [Airet]
19823,2167-02-25,STRENGTH-DRUG,STRENGTH,1539,1543,40 mg,DRUG,1551,1555,Lasix,1.0,Lasix 40 mg,0,Lasix 40 mg,null,200809,200809:::617319:::103919:::1871459:::201286:::2556796:::1927858:::1648194:::977916:::352320:::208458:::1738528:::2548764:::2100009:::104437:::152923:::1040034:::1307305:::2167570:::152944:::1734953:::200802:::1312404:::1652065:::897742,furosemide 40 MG Oral Tablet [Lasix]:::atorvastatin 40 MG [Lipitor]:::fluvastatin 40 MG Oral Capsule [Lescol]:::lisdexamfetamine dimesylate 40 MG Chewable Tablet [Vyvanse]:::verapamil hydrochloride 40 MG Oral Tablet [Cordilox]:::relugolix 40 MG:::betrixaban 40 MG [Bevyxxa]:::methylphenidate hydrochloride 40 MG [Aptensio]:::oxymorphone hydrochloride 40 MG [Opana]:::atomoxetine 40 MG Oral Capsule [Strattera]:::tacrine 40 MG Oral Capsule [Cognex]:::paroxetine mesylate 40 MG [Pexeva]:::selinexor 40 MG [Xpovio]:::baloxavir marboxil 40 MG [Xofluza]:::moxisylyte 40 MG Oral Tablet [Opilon]:::simvastatin 40 MG Oral Tablet [Zocor]:::lurasidone hydrochloride 40 MG [Latuda]:::enzalutamide 40 MG [Xtandi]:::rosuvastatin 40 MG [Ezallor]:::stavudine 40 MG Oral Capsule [Zerit]:::24 HR methylphenidate 40 MG Chewable Extended Release Oral Tablet [QuilliChew]:::furosemide 40 MG Oral Tablet [Aluzine]:::regorafenib 40 MG [Stivarga]:::duloxetine 40 MG [Irenka]:::verapamil hydrochloride 40 MG [Calan]
19823,2167-02-25,ROUTE-DRUG,ROUTE,1548,1549,IV,DRUG,1551,1555,Lasix,1.0,Lasix,0,Lasix,null,202991,202991:::151963:::22569

In [0]:
rxnorm_result = rxnorm_result.withColumn('all_codes', F.split(F.col('all_codes'), ':::'))\
                             .withColumn('resolutions', F.split(F.col('resolutions'), ':::'))
rxnorm_result.limit(20).display()

subject_id,date,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence,rx_text,sent_id,ner_chunk,entity,rxnorm_code,all_codes,resolutions
19823,2167-02-25,DRUG-FORM,DRUG,1391,1399,Albuterol,FORM,1414,1423,nebulizers,1.0,Albuterol nebulizers,0,Albuterol nebulizers,null,2108226,"List(2108226, 1154602, 370790, 1154603, 2108233, 2108255, 2108276, 745678, 1163444, 2108246, 2108507, 1154987, 2108228, 437534, 1008406, 2108280, 2108463, 1154598, 2108451, 2108291, 2108552, 377207, 1154604, 1156070)","List(albuterol Inhalation Solution, albuterol Inhalant Product, albuterol Injectable Solution, albuterol Injectable Product, albuterol Inhalation Powder, metaproterenol Inhalation Solution [Alupent], arformoterol Inhalation Solution, albuterol Metered Dose Inhaler, levalbuterol Inhalant Product, albuterol Inhalation Solution [Airet], sodium chloride Inhalation Solution [Nebusal], atropine Inhalant Product, albuterol Inhalation Solution [Accuneb], albuterol / ambroxol Oral Solution, Albuterol / Ambroxol, atropine Inhalation Solution, isoflurane Inhalation Solution [Attane], albuterol / ipratropium Inhalant Product, ipratropium Inhalation Solution [Atrovent], bitolterol Inhalation Solution, tobramycin Inhalation Solution [Tobi], acebutolol Injectable Solution, albuterol Oral Liquid Product, budesonide Inhalant Product)"
19823,2167-02-25,DRUG-FORM,DRUG,1405,1412,Atrovent,FORM,1414,1423,nebulizers,1.0,Atrovent nebulizers,0,Atrovent nebulizers,null,2108451,"List(2108451, 1173573, 379767, 1173576, 2463732, 1945043, 1172634, 1171309, 363357, 1184866, 1170108, 1593875, 1941535, 1547449, 1547653, 2108255, 1367507, 2056699, 1795589, 2108507, 1170006, 1170113, 1176096, 2108246)","List(ipratropium Inhalation Solution [Atrovent], Atrovent Inhalant Product, Atrovent Autohaler, Atrovent Nasal Product, epoetin alfa Injectable Solution [Retacrit], Trelegy Inhalant Product, Fortical Inhalant Product, Advair Inhalant Product, alefacept Injectable Solution [Amevive], Suprane Inhalant Product, Arduan Injectable Product, Altamist Inhalant Product, Armonair Inhalant Product, Nutrilyte Injectable Product, Arnuity Inhalant Product, metaproterenol Inhalation Solution [Alupent], Adasuve Inhalant Product, Ajovy Injectable Product, Nasoflow Inhalant Product, sodium chloride Inhalation Solution [Nebusal], Alupent Inhalant Product, Arfonad Injectable Product, Haldol Injectable Product, albuterol Inhalation Solution [Airet])"
19823,2167-02-25,STRENGTH-DRUG,STRENGTH,1539,1543,40 mg,DRUG,1551,1555,Lasix,1.0,Lasix 40 mg,0,Lasix 40 mg,null,200809,"List(200809, 617319, 103919, 1871459, 201286, 2556796, 1927858, 1648194, 977916, 352320, 208458, 1738528, 2548764, 2100009, 104437, 152923, 1040034, 1307305, 2167570, 152944, 1734953, 200802, 1312404, 1652065, 897742)","List(furosemide 40 MG Oral Tablet [Lasix], atorvastatin 40 MG [Lipitor], fluvastatin 40 MG Oral Capsule [Lescol], lisdexamfetamine dimesylate 40 MG Chewable Tablet [Vyvanse], verapamil hydrochloride 40 MG Oral Tablet [Cordilox], relugolix 40 MG, betrixaban 40 MG [Bevyxxa], methylphenidate hydrochloride 40 MG [Aptensio], oxymorphone hydrochloride 40 MG [Opana], atomoxetine 40 MG Oral Capsule [Strattera], tacrine 40 MG Oral Capsule [Cognex], paroxetine mesylate 40 MG [Pexeva], selinexor 40 MG [Xpovio], baloxavir marboxil 40 MG [Xofluza], moxisylyte 40 MG Oral Tablet [Opilon], simvastatin 40 MG Oral Tablet [Zocor], lurasidone hydrochloride 40 MG [Latuda], enzalutamide 40 MG [Xtandi], rosuvastatin 40 MG [Ezallor], stavudine 40 MG Oral Capsule [Zerit], 24 HR methylphenidate 40 MG Chewable Extended Release Oral Tablet [QuilliChew], furosemide 40 MG Oral Tablet [Aluzine], regorafenib 40 MG [Stivarga], duloxetine 40 MG [Irenka], verapamil hydrochloride 40 MG [Calan])"
19823,2167-02-25,ROUTE-DRUG,ROUTE,1548,1549,IV,DRUG,1551,1555,Lasix,1.0,Lasix,0,Lasix,null,202991,"List(202991, 151963, 2256936, 2256930, 1043720, 224946, 217961, 203783, 261550, 1013021, 606658, 218019, 1364397, 

### Split Resolutions to Resolution Drug and Write Results to Golden Delta Layer

In [0]:
pd_rxnorm_result = rxnorm_result.toPandas()
pd_rxnorm_result

Out[15]:

,subject_id,date,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence,rx_text,sent_id,ner_chunk,entity,rxnorm_code,all_codes,resolutions
0,19823,2167-02-25,DRUG-FORM,DRUG,1391,1399,Albuterol,FORM,1414,1423,nebulizers,1.0,Albuterol nebulizers,0,Albuterol nebulizers,None,2108226,"[2108226, 1154602, 370790, 1154603, 2108233, 2108255, 2108276, 745678, 1163444, 2108246, 2108507...","[albuterol Inhalation Solution, albuterol Inhalant Product, albuterol Injectable Solution, albut..."
1,19823,2167-02-25,DRUG-FORM,DRUG,1405,1412,Atrovent,FORM,1414,1423,nebulizers,1.0,Atrovent nebulizers,0,Atrovent nebulizers,None,2108451,"[2108451, 1173573, 379767, 1173576, 2463732, 1945043, 1172634, 1171309, 363357, 1184866, 1170108...","[ipratropium Inhalation Solution [Atrovent], Atrovent Inhalant Product, Atrovent Autohaler, Atro..."
2,19823,2167-02-25,STRENGTH-DRUG,STRENGTH,1539,1543,40 mg,DRUG,1551,1555,Lasix,1.0,Lasix 40 mg,0,Lasix 40 mg,None,200809,"[200809, 617319, 103919, 1871459, 201286, 2556796, 1927858, 1648194, 977916, 352320, 208458, 173...","[furosemide 40 MG Oral Tablet [Lasix], atorvastatin 40 MG [Lipitor], fluvastatin 40 MG Oral Caps..."
3,19823,2167-02-25,ROUTE-DRUG,ROUTE,1548,1549,IV,DRUG,1551,1555,Lasix,1.0,Lasix,0,Lasix,None,202991,"[202991, 151963, 2256936, 2256930, 1043720, 224946, 217961, 203783, 261550, 1013021, 606658, 218...","[Lasix, Lasma, lasmiditan Oral Tablet, lasmiditan, LidoWorx, Lidex, Laniroif, Lanoxicaps, Lanabi..."
4,19823,2167-02-25,DRUG-STRENGTH,DRUG,2336,2341,Amaryl,STRENGTH,2343,2348,2.0 mg,1.0,Amaryl 2.0 mg,0,Amaryl 2.0 mg,None,901295,"[901295, 153591, 1310138, 213799, 2399657, 1036818, 998190, 1439900, 905270, 540140, 202295, 104...","[sodium fluoride 2.2 MG [Ludent], glimepiride 2 MG Oral Tablet [Amaryl], everolimus 2 MG Tablet ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2777,70004,2182-08-05,ROUTE-DRUG,ROUTE,545,546,IV,DRUG,548,555,contrast,1.0,contrast,0,contrast,None,1592743,"[1592743, 202939, 23202, 66977, 2262255, 543455, 705766, 1436150, 744843, 216795, 1946584, 65874...","[Ofev, Dixarit, Dilor, Vascor, Scenesse, Durad, Appearex, Visco-Gel, Isentress, Duratest, Xhance..."
2778,70004,2182-08-05,DOSAGE-DRUG,DOSAGE,942,946,20 cc,DRUG,951,959,Magnevist,1.0,Magnevist 20 cc,0,Magnevist 20 cc,None,208456,"[208456, 152893, 2286257, 617317, 664142, 1119558, 596927, 429343, 440810, 571777, 351387, 79386...","[tacrine 20 MG Oral Capsule [Cognex], sertindole 20 MG Oral Tablet [Serdolect], dexamethasone 20..."
2779,70004,2182-08-05,DRUG-ROUTE,DRUG,951,959,Magnevist,ROUTE,961,971,intravenous,1.0,Magnevist,0,Magnevist,None,196214,"[196214, 2475179, 406156, 991881, 6574, 218204, 218250, 218167, 797858, 1043619, 152000, 218245,...","[Magnesiocard, magnesite, MagneBind, Maracyn Plus, magnesium, Maoson, Maxaquin, Magagel Plus, Ma..."
2780,70004,2182-08-16,ROUTE-DRUG,ROUTE,475,476,IV,DRUG,478,485,CONTRAST,1.0,CONTRAST,0,CONTRAST,None,799044,"[799044, 153381, 385716, 216281, 668395, 1013644, 216253, 284702, 1188463, 2264346, 323984, 2158...","[Cotab A, Cozaar-Comp, Cesamet, Crolom, Certuss, Cidaflex, Cosopt, Colocort, Citravet, belladonn..."


In [0]:
pd_rxnorm_result['drug_resolution']= pd_rxnorm_result['resolutions'].apply(lambda x: x[0])
pd_rxnorm_result['drug_resolution'] = pd_rxnorm_result['drug_resolution'].str.lower()
pd_rxnorm_result['chunk1']          = pd_rxnorm_result['chunk1'].str.lower()
pd_rxnorm_result['chunk2']          = pd_rxnorm_result['chunk2'].str.lower()
pd_rxnorm_result.head(4)

Out[16]:

,subject_id,date,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence,rx_text,sent_id,ner_chunk,entity,rxnorm_code,all_codes,resolutions,drug_resolution
0,19823,2167-02-25,DRUG-FORM,DRUG,1391,1399,albuterol,FORM,1414,1423,nebulizers,1.0,Albuterol nebulizers,0,Albuterol nebulizers,None,2108226,"[2108226, 1154602, 370790, 1154603, 2108233, 2108255, 2108276, 745678, 1163444, 2108246, 2108507...","[albuterol Inhalation Solution, albuterol Inhalant Product, albuterol Injectable Solution, albut...",albuterol inhalation solution
1,19823,2167-02-25,DRUG-FORM,DRUG,1405,1412,atrovent,FORM,1414,1423,nebulizers,1.0,Atrovent nebulizers,0,Atrovent nebulizers,None,2108451,"[2108451, 1173573, 379767, 1173576, 2463732, 1945043, 1172634, 1171309, 363357, 1184866, 1170108...","[ipratropium Inhalation Solution [Atrovent], Atrovent Inhalant Product, Atrovent Autohaler, Atro...",ipratropium inhalation solution [atrovent]
2,19823,2167-02-25,STRENGTH-DRUG,STRENGTH,1539,1543,40 mg,DRUG,1551,1555,lasix,1.0,Lasix 40 mg,0,Lasix 40 mg,None,200809,"[200809, 617319, 103919, 1871459, 201286, 2556796, 1927858, 1648194, 977916, 352320, 208458, 173...","[furosemide 40 MG Oral Tablet [Lasix], atorvastatin 40 MG [Lipitor], fluvastatin 40 MG Oral Caps...",furosemide 40 mg oral tablet [lasix]
3,19823,2167-02-25,ROUTE-DRUG,ROUTE,1548,1549,iv,DRUG,1551,1555,lasix,1.0,Lasix,0,Lasix,None,202991,"[202991, 151963, 2256936, 2256930, 1043720, 224946, 217961, 203783, 261550, 1013021, 606658, 218...","[Lasix, Lasma, lasmiditan Oral Tablet, lasmiditan, LidoWorx, Lidex, Laniroif, Lanoxicaps, Lanabi...",lasix


In [0]:
outname = 'posology_RE_rxnorm_w_drug_resolutions.csv'
outdir = f'/FileStore/HLS/jsl_kg/data/'

In [0]:
pd_rxnorm_result.to_csv(f'/dbfs{outdir+outname}', index=False, encoding="utf-8")

## NER JSL Slim

Model card of the ner_jsl_slim is [here](https://nlp.johnsnowlabs.com/2021/08/13/ner_jsl_slim_en.html).

In [0]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

sentenceDetector = SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")\
      .setCustomBounds(["\|"])

tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")\

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

jsl_ner = MedicalNerModel.pretrained("ner_jsl_slim", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

jsl_converter = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")\
      .setWhiteList(['Symptom','Body_Part', 'Procedure', 'Disease_Syndrome_Disorder', 'Test'])

ner_pipeline = Pipeline(
    stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        jsl_ner,
        jsl_converter
        ])

data_ner = spark.createDataFrame([[""]]).toDF("text")
model = ner_pipeline.fit(data_ner)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_jsl_slim download started this may take some time.
[ | ][ / ][ — ][OK!]

In [0]:
results = model.transform(df)
results.printSchema()

root
-- subject_id: long (nullable = true)
-- date: string (nullable = true)
-- text: string (nullable = true)
-- gender: string (nullable = true)
-- dateOfBirth: string (nullable = true)
-- document: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- sentence: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- token: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- embeddings: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- ner: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- ner_chunk: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)

In [0]:
result_df = results.select('subject_id','date',
                           F.explode(F.arrays_zip(results.ner_chunk.result, results.ner_chunk.begin, results.ner_chunk.end, results.ner_chunk.metadata)).alias("cols")) \
                    .select('subject_id','date',
                            F.expr("cols['3']['sentence']").alias("sentence_id"),
                            F.expr("cols['0']").alias("chunk"),
                            F.expr("cols['1']").alias("begin"),
                            F.expr("cols['2']").alias("end"),
                            F.expr("cols['3']['entity']").alias("ner_label"))\
                    .filter("ner_label!='O'")
result_df.limit(20).display()

subject_id,date,sentence_id,chunk,begin,end,ner_label
19823,2167-02-25,0,Shortness of breath,178,196,Symptom
19823,2167-02-25,0,cough,199,203,Symptom
19823,2167-02-25,1,diabetes type II,345,360,Disease_Syndrome_Disorder
19823,2167-02-25,1,congestive heart failure,363,386,Disease_Syndrome_Disorder
19823,2167-02-25,1,hypertension,413,424,Disease_Syndrome_Disorder
19823,2167-02-25,2,progressively worsening shortness of breath,477,519,Symptom
19823,2167-02-25,2,dyspnea on exertion,525,543,Symptom
19823,2167-02-25,2,wheezing,546,553,Symptom
19823,2167-02-25,2,nonproductive cough,556,574,Symptom
19823,2167-02-25,3,nausea,666,671,Symptom


In [0]:
pd_result = result_df.toPandas()
pd_result

Out[22]:

,subject_id,date,sentence_id,chunk,begin,end,ner_label
0,19823,2167-02-25,0,Shortness of breath,178,196,Symptom
1,19823,2167-02-25,0,cough,199,203,Symptom
2,19823,2167-02-25,1,diabetes type II,345,360,Disease_Syndrome_Disorder
3,19823,2167-02-25,1,congestive heart failure,363,386,Disease_Syndrome_Disorder
4,19823,2167-02-25,1,hypertension,413,424,Disease_Syndrome_Disorder
...,...,...,...,...,...,...,...
16516,70004,2182-08-16,12,cerebellum,1812,1821,Body_Part
16517,70004,2182-08-16,13,Multilevel degenerative changes,1860,1890,Symptom
16518,70004,2182-08-16,13,uncovertebral joint hypertrophy,1897,1927,Disease_Syndrome_Disorder
16519,70004,2182-08-16,14,cervical spine,1992,2005,Body_Part


In [0]:
outname = 'ner_jsl_slim_results.csv'
outdir = f'/FileStore/HLS/jsl_kg/data/'
pd_result.to_csv(f'/dbfs{outdir+outname}', index=False, encoding="utf-8")

## License
Copyright / License info of the notebook. Copyright [2021] the Notebook Authors.  The source in this notebook is provided subject to the [Apache 2.0 License](https://spdx.org/licenses/Apache-2.0.html).  All included or referenced third party libraries are subject to the licenses set forth below.

|Library Name|Library License|Library License URL|Library Source URL|
| :-: | :-:| :-: | :-:|
|Pandas |BSD 3-Clause License| https://github.com/pandas-dev/pandas/blob/master/LICENSE | https://github.com/pandas-dev/pandas|
|Numpy |BSD 3-Clause License| https://github.com/numpy/numpy/blob/main/LICENSE.txt | https://github.com/numpy/numpy|
|Apache Spark |Apache License 2.0| https://github.com/apache/spark/blob/master/LICENSE | https://github.com/apache/spark/tree/master/python/pyspark|
|BeautifulSoup|MIT License|https://www.crummy.com/software/BeautifulSoup/#Download|https://www.crummy.com/software/BeautifulSoup/bs4/download/|
|Requests|Apache License 2.0|https://github.com/psf/requests/blob/main/LICENSE|https://github.com/psf/requests|
|Spark NLP Display|Apache License 2.0|https://github.com/JohnSnowLabs/spark-nlp-display/blob/main/LICENSE|https://github.com/JohnSnowLabs/spark-nlp-display|
|Spark NLP |Apache License 2.0| https://github.com/JohnSnowLabs/spark-nlp/blob/master/LICENSE | https://github.com/JohnSnowLabs/spark-nlp|
|Spark NLP for Healthcare|[Proprietary license - John Snow Labs Inc.](https://www.johnsnowlabs.com/spark-nlp-health/) |NA|NA|




|Author|
|-|
|Databricks Inc.|
|John Snow Labs Inc.|

## Disclaimers
Databricks Inc. (“Databricks”) does not dispense medical, diagnosis, or treatment advice. This Solution Accelerator (“tool”) is for informational purposes only and may not be used as a substitute for professional medical advice, treatment, or diagnosis. This tool may not be used within Databricks to process Protected Health Information (“PHI”) as defined in the Health Insurance Portability and Accountability Act of 1996, unless you have executed with Databricks a contract that allows for processing PHI, an accompanying Business Associate Agreement (BAA), and are running this notebook within a HIPAA Account.  Please note that if you run this notebook within Azure Databricks, your contract with Microsoft applies.